In [4]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
tf.random.set_seed(10)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt

2022-10-15 15:45:40.931215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 15:45:43.010125: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 15:45:49.927948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-15 15:45:49.928042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [5]:
import glob

In [6]:
# In physical space use 'raw_train.reshape(-1,121,281)' to visualize original
num_snapshots = 12564
directory = '/lcrc/project/AIEADA-2/era5_data/full_data/snapshots/'

#var_list = [raw_train_z500,
#            raw_train_u250,raw_train_v250,raw_train_t250,
#            raw_train_u850,raw_train_v850,raw_train_t850,
#            raw_train_blh,raw_train_tcwv]
input_window = 14
output_window = 7

In [7]:
def load_file_npy(file_path):
    file_path_string = bytes.decode(file_path.numpy())
    data = np.load(file_path_string)
    return data

In [8]:
def read_npy_file(file_path):
    file_path_string = bytes.decode(file_path.numpy())
    data = np.load(file_path_string)
    return data.astype(np.float32)

In [33]:
def parse_function(file_path):
    #example = tf.py_function(load_file_npy, inp=file_path, Tout=tf.float32)
    example = tf.py_function(read_npy_file, [file_path], tf.float32)
    #example = tf.py_function(np.load, file_path, tf.string)
    #tf.print(file_path)
    #example = tf.io.read_file(file_path)
    #example = tf.io.decode_raw(example, tf.float32)
    print(example)
    input_window = 14
    input_data = example[:input_window,:]
    output_data = example[input_window:,:]
    #return input_data, {'pred':output_data, 'recon':input_data}
    return input_data, (output_data, input_data)

In [34]:
def build_dataset(batch_size):
    dataset_dir = directory + 'split_examples/train_data/'
    filelist = glob.glob(dataset_dir + '*.npy')
    train_dataset = tf.data.Dataset.from_tensor_slices(filelist)
    shuffle_buffer = 6000
    train_dataset = train_dataset.shuffle(shuffle_buffer,reshuffle_each_iteration=True)
    train_dataset = train_dataset.map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    #train_dataset = train_dataset.map(
    #    lambda item: tuple(tf.py_function(read_npy_file, [item], [tf.float32,])))
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return train_dataset

In [35]:
example = np.load(directory + 'split_examples/train_data/example151.npy')

In [36]:
print(example.shape)
print(example.dtype)

(21, 38979)
float32


In [37]:
batch_size = 512
train_dataset = build_dataset(batch_size)

Tensor("EagerPyFunc:0", dtype=float32, device=/job:localhost/replica:0/task:0)


In [38]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)))>

In [39]:
embed_dim = example.shape[-1]
output_dim = embed_dim
encode_dim = 180

In [40]:
num_ae_encoder_layers = 3
num_ae_decoder_layers = 3
num_lstm_cells_encoder = 2
num_lstm_cells_decoder = 2

In [41]:
ff_dim = 100  # Hidden layer size in feed forward network inside transformer
dropout_rate = 0.0

inputs = layers.Input(shape=(input_window,embed_dim))
ae_encoding_layers = []
for _ in range(num_ae_encoder_layers):
    ae_encoding_layers.append(layers.TimeDistributed(layers.Dense(encode_dim,activation='elu')))

lstm_encoder_cells = []
for _ in range(num_lstm_cells_encoder):
    lstm_encoder_cells.append(layers.Bidirectional(layers.LSTM(ff_dim,activation='elu',return_sequences=True,)))

lstm_encoder_final = layers.Bidirectional(layers.LSTM(ff_dim,activation='elu'))
lstm_repeater_layer = layers.RepeatVector(output_window)
    
lstm_decoder_cells = []
for _ in range(num_lstm_cells_decoder):
    lstm_decoder_cells.append(layers.LSTM(encode_dim,activation='elu',return_sequences=True,))

    
ae_decoding_layers = []
for _ in range(num_ae_decoder_layers):
    ae_decoding_layers.append(layers.TimeDistributed(layers.Dense(embed_dim,activation='elu')))


# Encode from physical space
print('Input shape:',inputs.get_shape().as_list())

x = inputs
for i in range(num_ae_encoder_layers):
    x = ae_encoding_layers[i](x)
encoded = x

print('AE Encoded shape:',encoded.get_shape().as_list())

x = lstm_encoder_cells[0](encoded)    
for i in range(1,num_lstm_cells_encoder):
    x = lstm_encoder_cells[i](x)
    
x = lstm_encoder_final(x)

print('LSTM Encoded shape:',x.get_shape().as_list())

x = lstm_repeater_layer(x)

for i in range(num_lstm_cells_decoder):
    x = lstm_decoder_cells[i](x)

print('LSTM Decoded shape:',x.get_shape().as_list())
    
for i in range(num_ae_decoder_layers):
    x = ae_decoding_layers[i](x)
    
outputs = x

print('AE+LSTM Output shape:',outputs.get_shape().as_list())
    
decoded = encoded
for i in range(num_ae_decoder_layers):
    decoded = ae_decoding_layers[i](decoded)
    
print('AE Output shape:',decoded.get_shape().as_list())

model = tf.keras.Model(inputs=inputs, outputs=[outputs,decoded])

Input shape: [None, 14, 38979]
AE Encoded shape: [None, 14, 180]
LSTM Encoded shape: [None, 200]
LSTM Decoded shape: [None, 7, 180]
AE+LSTM Output shape: [None, 7, 38979]
AE Output shape: [None, 14, 38979]


In [42]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss',patience=20)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='mean_squared_error',loss_weights=[1,1])

In [43]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 14, 38979)]  0           []                               
                                                                                                  
 time_distributed_12 (TimeDistr  (None, 14, 180)     7016400     ['input_3[0][0]']                
 ibuted)                                                                                          
                                                                                                  
 time_distributed_13 (TimeDistr  (None, 14, 180)     32580       ['time_distributed_12[0][0]']    
 ibuted)                                                                                          
                                                                                            

In [ ]:
history = model.fit(train_dataset,epochs=250,batch_size=2,callbacks=[reduce_lr,early_stop])

Epoch 1/250
 4/25 [===>..........................] - ETA: 1:07:59 - loss: 0.4723 - time_distributed_17_loss: 0.2644 - time_distributed_17_1_loss: 0.2078